Deep Learning
=============

CNN's
------------

Previously we trained fully connected networks to classify notMNIST  (http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters. The goal here is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
# Original model 
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.990098
Minibatch accuracy: 18.8%
Validation accuracy: 12.3%
Minibatch loss at step 200: 0.973794
Minibatch accuracy: 62.5%
Validation accuracy: 76.8%
Minibatch loss at step 400: 0.358453
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 600: 0.750550
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
Minibatch loss at step 800: 0.980536
Minibatch accuracy: 68.8%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 0.710776
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 1200: 0.902813
Minibatch accuracy: 68.8%
Validation accuracy: 83.9%
Minibatch loss at step 1400: 0.313149
Minibatch accuracy: 93.8%
Validation accuracy: 84.4%
Minibatch loss at step 1600: 0.241994
Minibatch accuracy: 93.8%
Validation accuracy: 84.7%
Minibatch loss at step 1800: 0.242745
Minibatch accuracy: 93.8%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.358230
Minibatch accuracy: 93.8%
Validation accuracy:

---
Max Pooling
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
# using Max pooling
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # Notice here that the     
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    print('Shape of data,conv is:',  tf.Tensor.get_shape(data),tf.Tensor.get_shape(conv))
    hidden = tf.nn.relu(conv + layer1_biases)
    print('Shape of hidden 1, is:',  tf.Tensor.get_shape(hidden))
    mpool1 = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    print('Shape of maxpool1, is:',  tf.Tensor.get_shape(mpool1))
    conv = tf.nn.conv2d(mpool1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print('Shape of conv 2, is:',  tf.Tensor.get_shape(conv))
    hidden = tf.nn.relu(conv + layer2_biases)
    #mpool2 = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    print('Shape of hidden 2, is:',  tf.Tensor.get_shape(hidden))
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('Shape of hidden 3, is:',  tf.Tensor.get_shape(hidden))
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

('Shape of data,conv is:', TensorShape([Dimension(64), Dimension(28), Dimension(28), Dimension(1)]), TensorShape([Dimension(64), Dimension(28), Dimension(28), Dimension(16)]))
('Shape of hidden 1, is:', TensorShape([Dimension(64), Dimension(28), Dimension(28), Dimension(16)]))
('Shape of maxpool1, is:', TensorShape([Dimension(64), Dimension(14), Dimension(14), Dimension(16)]))
('Shape of conv 2, is:', TensorShape([Dimension(64), Dimension(7), Dimension(7), Dimension(16)]))
('Shape of hidden 2, is:', TensorShape([Dimension(64), Dimension(7), Dimension(7), Dimension(16)]))
('Shape of hidden 3, is:', TensorShape([Dimension(64), Dimension(64)]))
('Shape of data,conv is:', TensorShape([Dimension(10000), Dimension(28), Dimension(28), Dimension(1)]), TensorShape([Dimension(10000), Dimension(28), Dimension(28), Dimension(16)]))
('Shape of hidden 1, is:', TensorShape([Dimension(10000), Dimension(28), Dimension(28), Dimension(16)]))
('Shape of maxpool1, is:', TensorShape([Dimension(10000), Dimen

In [ ]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
LeNet5
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [11]:
# The following structure (after 100000 steps) gives 96.1% accuracy.

batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # LeNet Architecture INPUT => CONV => RELU => POOL => CONV => RELU => POOL => FC => RELU => FC
  # Model.
  def model(data,kp):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    #print('Shape of data,conv is:',  tf.Tensor.get_shape(data),tf.Tensor.get_shape(conv))
    relu1 = tf.nn.relu(conv1 + layer1_biases)
    #print('Shape of relu1 1, is:',  tf.Tensor.get_shape(relu1))
    mpool1 = tf.nn.max_pool(relu1,[1,2,2,1],[1,2,2,1],padding='SAME')
    #print('Shape of maxpool1, is:',  tf.Tensor.get_shape(mpool1))
    conv2= tf.nn.conv2d(mpool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    #print('Shape of conv 2, is:',  tf.Tensor.get_shape(conv2))
    relu2 = tf.nn.relu(conv2+ layer2_biases)
    mpool2 = tf.nn.max_pool(relu2,[1,2,2,1],[1,2,2,1],padding='SAME')
    #relu3 = tf.nn.relu(mpool2+layer3_biases)
    #print('Shape of hidden 2, is:',  tf.Tensor.get_shape(mpool2))
    shape = mpool2.get_shape().as_list()
    reshape = tf.reshape(mpool2, [shape[0], shape[1] * shape[2] * shape[3]])
    relu3 = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),kp)
    #print('Shape of hidden 3, is:',  tf.Tensor.get_shape(hidden))
    return tf.matmul(relu3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset,0.75)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Learning rate decay
  #global_step = tf.Variable(0, trainable=False)
  #starter_learning_rate = 0.05
  #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
  #                                         500, 0.95, staircase=True)
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1))
  test_prediction = tf.nn.softmax(model(tf_test_dataset,1))

In [12]:
num_steps = 2001

with tf.Session(graph=graph,config = tf.ConfigProto(log_device_placement=True)) as session:
  with tf.device("/gpu:0"):  
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)      
      if (step % 500 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        print('Validation accuracy: %.1f%%' % accuracy(
          valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

InvalidArgumentError: Cannot assign a device to node 'init': Could not satisfy explicit device specification '/device:GPU:0' because no devices matching that specification are registered in this process; available devices: /job:localhost/replica:0/task:0/cpu:0
	 [[Node: init = NoOp[_device="/device:GPU:0"](^Variable/Assign, ^Variable_1/Assign, ^Variable_2/Assign, ^Variable_3/Assign, ^Variable_4/Assign, ^Variable_5/Assign, ^Variable_6/Assign, ^Variable_7/Assign, ^beta1_power/Assign, ^beta2_power/Assign, ^Variable/Adam/Assign, ^Variable/Adam_1/Assign, ^Variable_1/Adam/Assign, ^Variable_1/Adam_1/Assign, ^Variable_2/Adam/Assign, ^Variable_2/Adam_1/Assign, ^Variable_3/Adam/Assign, ^Variable_3/Adam_1/Assign, ^Variable_4/Adam/Assign, ^Variable_4/Adam_1/Assign, ^Variable_5/Adam/Assign, ^Variable_5/Adam_1/Assign, ^Variable_6/Adam/Assign, ^Variable_6/Adam_1/Assign, ^Variable_7/Adam/Assign, ^Variable_7/Adam_1/Assign)]]

Caused by op u'init', defined at:
  File "/usr/lib/python2.7/runpy.py", line 